In [1]:
from scenedetect import detect, AdaptiveDetector, split_video_ffmpeg
from pathlib import Path

raw_video_path = "/Users/scottcui/projects/mv_synthesis/datasets/raw/E01-Little_Quacker.mp4"
raw_video_path = Path(raw_video_path)
video_name = raw_video_path.stem
output_dir = raw_video_path.parent / "segments" / video_name

In [ ]:
detector = AdaptiveDetector(
    adaptive_threshold=1.4,
    min_scene_len=30,
    window_width=4,
    min_content_val=15,
)
scene_list = detect(str(raw_video_path), detector)
print(scene_list)
split_video_ffmpeg(str(raw_video_path), scene_list, output_dir=str(output_dir), show_progress=True)

<class 'scenedetect.frame_timecode.FrameTimecode'>


In [12]:
SEGMENT_DURATION_THRESHOLD = 3


def recursive_segment(video_path, start_seconds, end_seconds, adaptive_threshold=3.0, depth=0):
    detector = AdaptiveDetector(
        adaptive_threshold=adaptive_threshold,
        min_scene_len=30,
        window_width=16,
        min_content_val=10,
    )
    scene_list = detect(
        str(video_path),
        detector,
        show_progress=True,
        start_time=start_seconds,
        end_time=end_seconds,
    )
    final_scene_list = []
    for scene in scene_list:
        start_time, end_time = scene
        duration = end_time - start_time
        duration_seconds = duration.get_seconds()
        final_scene_list.append(scene)
        if duration_seconds > SEGMENT_DURATION_THRESHOLD and depth < 3:
            sub_scene_list = recursive_segment(
                video_path, start_time, end_time, adaptive_threshold * 0.8, depth + 1
            )
            final_scene_list.extend(sub_scene_list)
    return final_scene_list


scene_list = recursive_segment(raw_video_path, None, None, adaptive_threshold=2.5)
print(scene_list)

  Detected: 73 | Progress: 100%|██████████| 9555/9555 [00:38<00:00, 248.88frames/s]
  Detected: 0 | Progress: 100%|██████████| 116/116 [00:00<00:00, 247.99frames/s]
  Detected: 1 | Progress: 100%|██████████| 78/78 [00:00<00:00, 248.84frames/s]
  Detected: 0 | Progress: 100%|██████████| 339/339 [00:01<00:00, 243.02frames/s]
  Detected: 2 | Progress: 100%|██████████| 1429/1429 [00:05<00:00, 249.80frames/s]
  Detected: 0 | Progress: 100%|██████████| 517/517 [00:02<00:00, 249.78frames/s]
  Detected: 5 | Progress: 100%|██████████| 872/872 [00:03<00:00, 247.33frames/s]
  Detected: 0 | Progress: 100%|██████████| 334/334 [00:01<00:00, 245.15frames/s]
  Detected: 1 | Progress: 100%|██████████| 136/136 [00:00<00:00, 249.66frames/s]
  Detected: 3 | Progress: 100%|██████████| 273/273 [00:01<00:00, 249.79frames/s]
  Detected: 1 | Progress: 100%|██████████| 98/98 [00:00<00:00, 249.72frames/s]
  Detected: 0 | Progress: 100%|██████████| 96/96 [00:00<00:00, 248.59frames/s]
  Detected: 3 | Progress: 100

[(00:00:00.000 [frame=0, fps=22.355], 00:00:01.566 [frame=35, fps=22.355]), (00:00:01.566 [frame=35, fps=22.355], 00:00:06.755 [frame=151, fps=22.355]), (00:00:06.755 [frame=151, fps=22.355], 00:00:07.918 [frame=177, fps=22.355]), (00:00:07.918 [frame=177, fps=22.355], 00:00:11.407 [frame=255, fps=22.355]), (00:00:07.918 [frame=177, fps=22.355], 00:00:10.199 [frame=228, fps=22.355]), (00:00:10.199 [frame=228, fps=22.355], 00:00:11.407 [frame=255, fps=22.355]), (00:00:11.407 [frame=255, fps=22.355], 00:00:26.572 [frame=594, fps=22.355]), (00:00:26.572 [frame=594, fps=22.355], 00:00:27.645 [frame=618, fps=22.355]), (00:00:27.645 [frame=618, fps=22.355], 00:01:31.569 [frame=2047, fps=22.355]), (00:00:27.645 [frame=618, fps=22.355], 00:00:50.772 [frame=1135, fps=22.355]), (00:00:50.772 [frame=1135, fps=22.355], 00:01:29.780 [frame=2007, fps=22.355]), (00:00:50.772 [frame=1135, fps=22.355], 00:01:05.713 [frame=1469, fps=22.355]), (00:01:05.713 [frame=1469, fps=22.355], 00:01:07.995 [frame=1

In [14]:
def post_process_scene_list(scene_list):
    deduplicate_scene_list = sorted(list(set(scene_list)))
    processed = []
    for scene in deduplicate_scene_list:
        duration_seconds = scene[1].get_seconds() - scene[0].get_seconds()
        if duration_seconds >= 30 or duration_seconds < 1:
            continue
        processed.append(scene)
    
    print(f"Post-processed scene list length: {len(processed)} ({len(deduplicate_scene_list)} -> {len(processed)})")
    return processed

scene_list = post_process_scene_list(scene_list)
print(scene_list)

Post-processed scene list length: 178 (178 -> 178)
[(00:00:00.000 [frame=0, fps=22.355], 00:00:01.566 [frame=35, fps=22.355]), (00:00:01.566 [frame=35, fps=22.355], 00:00:06.755 [frame=151, fps=22.355]), (00:00:06.755 [frame=151, fps=22.355], 00:00:07.918 [frame=177, fps=22.355]), (00:00:07.918 [frame=177, fps=22.355], 00:00:10.199 [frame=228, fps=22.355]), (00:00:07.918 [frame=177, fps=22.355], 00:00:11.407 [frame=255, fps=22.355]), (00:00:10.199 [frame=228, fps=22.355], 00:00:11.407 [frame=255, fps=22.355]), (00:00:11.407 [frame=255, fps=22.355], 00:00:26.572 [frame=594, fps=22.355]), (00:00:26.572 [frame=594, fps=22.355], 00:00:27.645 [frame=618, fps=22.355]), (00:00:27.645 [frame=618, fps=22.355], 00:00:50.772 [frame=1135, fps=22.355]), (00:00:50.772 [frame=1135, fps=22.355], 00:01:05.713 [frame=1469, fps=22.355]), (00:01:05.713 [frame=1469, fps=22.355], 00:01:07.995 [frame=1520, fps=22.355]), (00:01:07.995 [frame=1520, fps=22.355], 00:01:12.468 [frame=1620, fps=22.355]), (00:01:07

In [15]:
split_video_ffmpeg(
    str(raw_video_path),
    scene_list,
    output_dir=str(output_dir),
    show_progress=True,
    output_file_template="$SCENE_NUMBER-$START_TIME-$END_TIME.mp4",
    arg_override="-map 0:v:0 -map 0:a? -map 0:s? -c:v h264_videotoolbox -quality 1 -b:v 2.5M -allow_sw 1 -c:a aac",
)

  0%|          | 0/9555 [00:00<?, ?frame/s]

16358frame [01:35, 172.05frame/s]                      


0